In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn import model_selection, linear_model, metrics
import statsmodels.stats.proportion as prop
import statsmodels.stats.weightstats as wsts

from proportions_hypothesis import *

/Users/artem/miniconda3/envs/mldaspec/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [87]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):
    """Confidence interval for two independent proportions
    
    """
    import scipy.stats
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    import scipy.stats
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    import scipy.stats
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    import scipy.stats
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    import scipy.stats
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

___
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [2]:
yawn = np.array([1]*10 + [0]*(34-10))
notyawn = np.array([1]*4 + [0]*(16-4))
#np.random.shuffle(yawn)
#np.random.shuffle(notyawn)

In [3]:
yawn.mean() - notyawn.mean()

0.04411764705882354

In [4]:
proportions_diff_z_test(
    proportions_diff_z_stat_ind(yawn, notyawn),
    alternative='greater')

0.37293045872523534

In [6]:
# import statsmodels.stats.proportion as prop
prop.proportions_ztest(
    [yawn.sum(), notyawn.sum()],
    [len(yawn), len(notyawn)],
    alternative='larger')

(0.32410186177608225, 0.37293045872523534)

___
Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:

In [24]:
df = pd.read_csv('data/banknotes.txt', sep='\t')
X = df.drop('real', axis=1)
y = df['real']
df.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [105]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=50, random_state=1)

In [106]:
X_train1 = X_train[['X1', 'X2', 'X3']]
X_test1 = X_test[['X1', 'X2', 'X3']]
X_train2 = X_train[['X4', 'X5', 'X6']]
X_test2 = X_test[['X4', 'X5', 'X6']]

In [116]:
lr_clf = linear_model.LogisticRegression()
lr_clf.fit(X_train1, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [117]:
preds1 = lr_clf.predict(X_test1)

In [118]:
lr_clf = linear_model.LogisticRegression()
lr_clf.fit(X_train2, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [119]:
preds2 = lr_clf.predict(X_test2)

In [120]:
errs1 = np.abs(y_test-preds1)

In [121]:
errs2 = np.abs(y_test-preds2)

In [122]:
proportions_diff_confint_rel(errs1, errs2)

(0.059945206279614305, 0.3000547937203857)

In [123]:
proportions_diff_z_test(proportions_diff_z_stat_rel(errs1, errs2))

0.0032969384555543435

___
Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [182]:
mu0 = 525
sigma = 100
n = 100
mu1 = 541.4
z1 = (mu1 - mu0) / (sigma / np.sqrt(n))
print(z1)

1.6399999999999977


In [183]:
scipy.stats.norm.sf(z1)

0.05050258347410395

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [184]:
mu2 = 541.5
z2 = (mu2 - mu0) / (sigma / np.sqrt(n))
print(z2)

1.65


In [185]:
scipy.stats.norm.sf(z2)

0.0494714680336481

#### statsmodels.stats.weightstats funcs:

##### zstat generic

In [186]:
wsts._zstat_generic(mu1, mu0, (sigma/n**0.5), alternative='larger')

(1.6399999999999977, 0.05050258347410395)

In [187]:
wsts._zstat_generic(mu2, mu0, (sigma/n**0.5), alternative='larger')

(1.65, 0.0494714680336481)

##### zconfint generic

In [178]:
np.array(wsts._zconfint_generic(mu1, sigma/np.sqrt(n), 0.05, 'larger')) - mu0

array([-0.04853627,         inf])

In [179]:
np.array(wsts._zconfint_generic(mu2, sigma/np.sqrt(n), 0.05, 'larger')) - mu0

array([0.05146373,        inf])